## NGSolve Model Templates
* A ready to use collection of physical models based on NGSolve
* Solid mechanics, Fluid Dynamics, Electromagnetics ...
* Easy to combine models to solve multi-physics problems
* Useful as basis for optimization, model order reduction, UQ, ...

... work in progress

## 2001 - 2015: C++ code only 
* A lot of special files and #ifdef SABINE
* Exchange C++ files and snippets, had to reconfigure and recompile 

## 2015 - xxxx: Python frontend
* Unified C++ code (with some extensions)
* Python formulation much shorter and expressive, good i-Tutorials
* Exchange Python files and snippets 
* Still a lot of expert knowledge required for advanced methods

import whatever we need 

In [1]:
import math, time

from ngsolve import *
import netgen.geom2d
import netgen.meshing
import netgen.gui

from ngsolve.internal import visoptions

from ngs_templates.Elasticity import * 
from ngs_templates.NavierStokes import *
from ngs_templates.Transport import *
from ngs_templates.ConvectionDiffusion import * 

print ("gui ist up")

gui ist up


# Elasticity Model

In [5]:
geo = netgen.geom2d.SplineGeometry()
geo.AddRectangle( (0, 0), (1, 0.1), bcs = ("bottom", "right", "top", "left"))

mesh = Mesh(geo.GenerateMesh(maxh=0.05))

model = Elasticity(mesh=mesh, materiallaw=HookeMaterial(E=200,nu=0.2),
                    dirichlet="left",
                    boundaryforce = { "right" : (0,0.2) },
                    nonlinear=True, order=4)

ngsglobals.msg_level = 0
model.Solve()

Draw (model.displacement, reset=True)
Draw (model.stress[0,0], mesh, "stress_xx")
SetVisualization (deformation=True)

In [6]:
dispright = Parameter(0)
model = Elasticity(mesh=mesh, materiallaw=HookeMaterial(E=200,nu=0.2), \
            boundarydisplacement = { "left" : (0,0), "right" : (0,dispright)},
            nonlinear=True, order=4)


Draw (model.displacement, reset=True) 
Draw (model.stress[0,0], mesh, "stress_xx")
SetVisualization (deformation=True)

In [25]:
dispright.Set (dispright.Get()+0.1)
 
ngsglobals.msg_level = 0
model.Solve()
Redraw()

In [ ]:
# mesh-file available from:
# https://nemesis.asc.tuwien.ac.at/index.php/s/fDdc6gn5bRRScJK

In [26]:
mesh = Mesh(netgen.meshing.ImportMesh("../examples/frame.unv"))
Draw(mesh)

In [27]:
model = Elasticity(mesh=mesh, materiallaw=HookeMaterial(200,0.2), \
                       dirichlet="holes",
                       boundaryforce = { "right" : (0,0,1), "left" : (0,0,-1) },
                       nonlinear=False, order=2)

with TaskManager():
    model.Solve()

Draw (model.displacement, mesh, "disp", reset=True)
Draw (model.stress, mesh, "stress")

SetVisualization (deformation=True)

myfes = H1(mesh, order=2)
normstress = GridFunction(myfes)
normstress.Set (Norm(model.stress))

Draw (normstress, mesh, "mises")
SetVisualization (min=0, max=20, deformation=True)

it =  0  err =  0.017747845128177966
it =  1  err =  0.002504689072149602
it =  2  err =  0.0004535117825175512
it =  3  err =  8.273675510240502e-05
it =  4  err =  1.9935081971500962e-05
it =  5  err =  5.500780277220284e-06
it =  6  err =  1.7156196848078284e-06
it =  7  err =  5.38807095090576e-07
it =  8  err =  1.6505441521335284e-07
it =  9  err =  4.589342075564507e-08
it =  10  err =  1.1819193369474404e-08
it =  11  err =  3.1575300107098543e-09
it =  12  err =  8.445729396426187e-10
it =  13  err =  2.1057097982218437e-10
it =  14  err =  5.7244138146335745e-11


# Navier Stokes model


In [29]:
geo = netgen.geom2d.SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl", maxh=0.02)
mesh = Mesh( geo.GenerateMesh(maxh=0.07))
Draw (mesh)
mesh.Curve(3)

In [30]:
timestep = 0.001
navstokes = NavierStokes (mesh, nu=0.001, order=4, timestep = timestep,
                              inflow="inlet", outflow="outlet", wall="wall|cyl",
                              uin=CoefficientFunction( (1.5*4*y*(0.41-y)/(0.41*0.41), 0) ))
                              

navstokes.SolveInitial()

Draw (navstokes.velocity, mesh, "velocity", reset=True)
Draw (navstokes.pressure, mesh, "pressure")

In [31]:
tend = 2
t = 0
with TaskManager():
    while t < tend:
        navstokes.DoTimeStep()
        t = t+timestep
        Redraw(blocking=False)

# Transport equation

In [32]:
mesh = Mesh( netgen.geom2d.unit_square.GenerateMesh(maxh=0.1))

timestep = 3e-3
transport = TransportEquation (mesh, order=6, wind = (y-0.5, -x+0.5), timestep=timestep)

transport.SetInitial( exp (-100* ((x-0.8)**2 + (y-0.5)**2) ) )

Draw (transport.concentration, mesh, "c", reset=True)
SetVisualization (min=0, max=1)

In [33]:
tend = 10
t = 0
with TaskManager(pajetrace=100*1000*1000):
    while t < tend:
        transport.DoTimeStep()
        t = t+timestep
        Redraw()

# Time-dependent Convection-Diffusion Equation


In [35]:
mesh = Mesh( netgen.geom2d.unit_square.GenerateMesh(maxh=0.1))

timestep = 10e-3
convdiff = ConvectionDiffusionEquation (mesh, order=3, lam=1e-3, wind = (y-0.5, -x+0.5), dirichlet=".*", timestep=timestep)

convdiff.SetInitial( exp (-100* ((x-0.8)**2 + (y-0.5)**2) ) )
Draw (convdiff.concentration, mesh, "c", reset=True)
SetVisualization (min=0, max=1)

In [36]:
tend = 10
t = 0
with TaskManager(pajetrace=100*1000*1000):
    while t < tend:
        convdiff.DoTimeStep()
        t = t+timestep
        Redraw()

# Natural Convection:

Coupling of Navier-Stokes and heat transport

* Change in temperatur leads to gravity forces
* Temperature is convected by fluid velocity

Rayleigh-Benard benchmark example

In [38]:
geo = netgen.geom2d.SplineGeometry()
geo.AddRectangle( (0,0), (0.06, 0.01), bcs=['b','r','t','l'])
mesh = Mesh(geo.GenerateMesh(maxh=0.002))
Draw (mesh)

In [39]:
timestep = 0.5
navstokes = NavierStokes (mesh, nu=1.04177e-6, order=3, timestep = timestep,
                              inflow="", outflow="", wall="l|r|b|t", uin=(0,0) )

Tinitial = 293.5-50*y+y*(0.01-y)*1e3*sin(20/0.06*x*math.pi)

convdiff = ConvectionDiffusionEquation (mesh, order=3, lam=1.38e-7, \
                wind = navstokes.velocity, dirichlet="b|t", udir=Tinitial, timestep=timestep)
convdiff.SetInitial(Tinitial)

T0 = 293
beta = 2.07e-4
navstokes.AddForce ( (1-beta*(convdiff.concentration-T0))*(0, -9.81))

navstokes.SolveInitial()

Draw (navstokes.pressure, mesh, "pressure", reset=True)
Draw (navstokes.velocity, mesh, "velocity")
visoptions.scalfunction='velocity:0'
Draw (convdiff.concentration, mesh, "temp")
input ("key")
t, tend = 0, 1000
with TaskManager(pajetrace=100*1000*1000):
    while t < tend:
        print (t)
        navstokes.DoTimeStep()
        convdiff.DoTimeStep()
        t = t+timestep
        Redraw()

keya
0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0
5.5
6.0
6.5
7.0
7.5
8.0
8.5
9.0
9.5
10.0
10.5
11.0
11.5
12.0
12.5
13.0
13.5
14.0
14.5
15.0
15.5
16.0
16.5
17.0
17.5
18.0
18.5
19.0
19.5
20.0
20.5
21.0
21.5
22.0
22.5
23.0
23.5
24.0
24.5
25.0
25.5
26.0
26.5
27.0
27.5
28.0
28.5
29.0
29.5
30.0
30.5
31.0
31.5
32.0
32.5
33.0
33.5
34.0
34.5
35.0
35.5
36.0
36.5
37.0
37.5
38.0
38.5
39.0
39.5
40.0
40.5
41.0
41.5
42.0
42.5
43.0
43.5
44.0
44.5
45.0
45.5
46.0
46.5
47.0
47.5
48.0
48.5
49.0
49.5
50.0
50.5
51.0
51.5
52.0
52.5
53.0
53.5
54.0
54.5
55.0
55.5
56.0
56.5
57.0
57.5
58.0
58.5
59.0
59.5
60.0
60.5
61.0
61.5
62.0
62.5
63.0
63.5
64.0
64.5
65.0
65.5
66.0
66.5
67.0
67.5
68.0
68.5
69.0
69.5
70.0
70.5
71.0
71.5
72.0
72.5
73.0
73.5
74.0
74.5
75.0
75.5
76.0
76.5
77.0
77.5
78.0
78.5
79.0
79.5
80.0
80.5
81.0
81.5
82.0
82.5
83.0
83.5
84.0
84.5
85.0
85.5
86.0
86.5
87.0
87.5
88.0
88.5
89.0
89.5
90.0
90.5
91.0
91.5
92.0
92.5
93.0
93.5
94.0
94.5
95.0
95.5
96.0
96.5
97.0
97.5
98.0
98.5
99.0
99.5
100.0
100.5
101.0

702.0
702.5
703.0
703.5
704.0
704.5
705.0
705.5
706.0
706.5
707.0
707.5
708.0
708.5
709.0
709.5
710.0
710.5
711.0
711.5
712.0
712.5
713.0
713.5
714.0
714.5
715.0
715.5
716.0
716.5
717.0
717.5
718.0
718.5
719.0
719.5
720.0
720.5
721.0
721.5
722.0
722.5
723.0
723.5
724.0
724.5
725.0
725.5
726.0
726.5
727.0
727.5
728.0
728.5
729.0
729.5
730.0
730.5
731.0
731.5
732.0
732.5
733.0
733.5
734.0
734.5
735.0
735.5
736.0
736.5
737.0
737.5
738.0
738.5
739.0
739.5
740.0
740.5
741.0
741.5
742.0
742.5
743.0
743.5
744.0
744.5
745.0
745.5
746.0
746.5
747.0
747.5
748.0
748.5
749.0
749.5
750.0
750.5
751.0
751.5
752.0
752.5
753.0
753.5
754.0
754.5
755.0
755.5
756.0
756.5
757.0
757.5
758.0
758.5
759.0
759.5
760.0
760.5
761.0
761.5
762.0
762.5
763.0
763.5
764.0
764.5
765.0
765.5
766.0
766.5
767.0
767.5
768.0
768.5
769.0
769.5
770.0
770.5
771.0
771.5
772.0
772.5
773.0
773.5
774.0
774.5
775.0
775.5
776.0
776.5
777.0
777.5
778.0
778.5
779.0
779.5
780.0
780.5
781.0
781.5
782.0
782.5
783.0
783.5
784.0
784.5
785.

# Have a look into some of the templates:
    
   * Elasticity 
   * ConverctionDiffusion 
   * NavierStokesSIMPLE

# Ongoing work:
- unified interfaces to user and solvers
- GUI integration  (--> Christopher Lackner)
- everything with ALE (--> Michael Neunteufel)
- same solvers in parallel  (--> Lukas Kogler)
- higher order time-stepping / space time 
- shape optimization ( --> Peter Gangl + Kevin Sturm)


# How to contribute ? 

- Try NGSolve Model Templates
 - Suggest missing features 
 - Suggest how to implement missing features
- field experts needed:
 - Cooperate on research on numerical methods like
   solvers, hpc, time-stepping, space-time, optimization, model order reduction
  
 - Cooperate on research in applications


# the end